# Curriculum Learning and LLC in Grokking (CORRECTED)

This notebook explores how curriculum learning (gradually increasing task difficulty with proper weight transfer) affects the Local Learning Coefficient (LLC) and grokking behavior in modular arithmetic.

**Based on the devinterp grokking example:**  
https://github.com/timaeus-research/devinterp/blob/main/examples/grokking.ipynb

## Key Corrections from Original Experiment:

1. ✅ **Equalized training budgets** - Direct training now gets same total batches as curriculum
2. ✅ **Proper weight transfer** - Pad/crop embedding and output layers instead of random reinitialization
3. ✅ **Multiple seeds** - Run 5 seeds with statistical significance testing
4. ✅ **3 chains for LLC** - Increased from 1 chain for reliability
5. ✅ **Fixed torch.argmax** - Added dimension parameter
6. ✅ **Conceptual clarity** - Acknowledged this is transfer learning between related tasks
7. ✅ **Adaptive LLC hyperparameters** - Scale by model size
8. ✅ **Statistical testing** - T-tests and confidence intervals

**Research Question:**  
Does sequential transfer learning through increasing moduli (mod-8 → mod-16 → mod-32 → mod-64) produce different LLC trajectories and generalization compared to direct training on mod-64?

**Important Note:**  
While we call this "curriculum learning," the tasks are related but not strictly hierarchical - mod-8 is not a subset of mod-16. This is more accurately described as **transfer learning on progressively complex related tasks**.

**Setup:**
- **Curriculum:** Train on mod-8 (50k), then mod-16 (50k), then mod-32 (50k), then mod-64 (50k) = 200k total
- **Direct:** Train directly on mod-64 for 200k batches (equalized budget!)
- **Seeds:** 5 independent runs (seeds 0-4)
- Track train/test accuracy, loss, and LLC at each stage

## 1. Import Required Libraries

In [ ]:
# Install required packages if running in a fresh environment
%pip install devinterp nbformat scipy

import random
from copy import deepcopy
from dataclasses import dataclass
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path
from scipy import stats

from devinterp.optim.sgld import SGLD
from devinterp.slt.sampler import estimate_learning_coeff_with_summary
from devinterp.utils import evaluate_ce

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Create results directory for saving figures
RESULTS_DIR = Path("../results/curriculum_learning_corrected")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Saving results to: {RESULTS_DIR.absolute()}")
print(f"Device: {DEVICE}")

# Utility function to save and show figures
def save_and_show(fig, filename, dpi=300):
    """Save figure to results directory and display it"""
    filepath = RESULTS_DIR / filename
    fig.savefig(filepath, dpi=dpi, bbox_inches='tight')
    print(f"Saved: {filepath.name}")
    plt.show()
    
print("Setup complete!")

## 2. Define Experiment Parameters

**Key Change:** Direct training gets 200k batches to match curriculum's total budget (4 stages × 50k)

In [ ]:
@dataclass
class ExperimentParams:
    n_batches_per_stage: int = 50000
    n_save_model_checkpoints: int = 100
    print_times: int = 100
    lr: float = 0.005
    batch_size: int = 128
    hidden_size: int = 48
    embed_dim: int = 12
    train_frac: float = 0.4
    device: str = DEVICE
    weight_decay: float = 0.0002

CURRICULUM_NUMS = [8, 16, 32, 64]
DIRECT_MODULUS = CURRICULUM_NUMS[-1]
SEEDS = [0, 1, 2, 3, 4]  # Run 5 seeds for statistical validity

# Equalized training budget
CURRICULUM_TOTAL_BATCHES = len(CURRICULUM_NUMS) * 50000  # 200k
DIRECT_TOTAL_BATCHES = CURRICULUM_TOTAL_BATCHES  # 200k (EQUALIZED!)

print(f"Curriculum stages: {CURRICULUM_NUMS}")
print(f"Curriculum total batches: {CURRICULUM_TOTAL_BATCHES:,}")
print(f"Direct total batches: {DIRECT_TOTAL_BATCHES:,}")
print(f"Seeds: {SEEDS}")
print(f"\n✅ Training budgets are now equalized!")

## 3. Define MLP Model and Utility Functions

**Key Changes:**
- Fixed `torch.argmax` to include dimension parameter
- Added proper weight transfer function with padding/cropping

In [ ]:
class MLP(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.embedding = nn.Embedding(params.p, params.embed_dim)
        self.linear1r = nn.Linear(params.embed_dim, params.hidden_size, bias=True)
        self.linear1l = nn.Linear(params.embed_dim, params.hidden_size, bias=True)
        self.linear2 = nn.Linear(params.hidden_size, params.p, bias=False)
        self.act = nn.GELU()
        self.vocab_size = params.p

    def forward(self, x):
        x = x.to(self.embedding.weight.device)
        x1 = self.embedding(x[..., 0])
        x2 = self.embedding(x[..., 1])
        x1 = self.linear1l(x1)
        x2 = self.linear1r(x2)
        x = x1 + x2
        x = self.act(x)
        x = self.linear2(x)
        return x

def test(model, dataset, device):
    """Test model on dataset. FIXED: torch.argmax now includes dim parameter."""
    n_correct = 0
    total_loss = 0
    model.eval()
    loss_fn = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in dataset:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = loss_fn(out, y)
            total_loss += loss.item()
            pred = torch.argmax(out, dim=-1)  # FIXED: Added dim=-1
            if pred == y:
                n_correct += 1
    return n_correct / len(dataset), total_loss / len(dataset)

def transfer_weights_with_padding(new_model, prev_model, verbose=False):
    """Transfer weights from prev_model to new_model, padding/cropping as needed.
    
    Key improvement: Instead of random reinitialization, we:
    - Pad embedding/output layers with random initialization for new dimensions
    - Crop if going to smaller model (though we don't in curriculum)
    - Fully transfer matching layers
    """
    new_dict = new_model.state_dict()
    prev_dict = prev_model.state_dict()
    
    transferred_params = 0
    total_params = 0
    
    for k in new_dict:
        total_params += new_dict[k].numel()
        
        if k not in prev_dict:
            if verbose:
                print(f"  {k}: Not in prev model, keeping random init")
            continue
            
        new_shape = new_dict[k].shape
        prev_shape = prev_dict[k].shape
        
        # Exact match - full transfer
        if new_shape == prev_shape:
            new_dict[k] = prev_dict[k].clone()
            transferred_params += new_dict[k].numel()
            if verbose:
                print(f"  {k}: Full transfer {prev_shape}")
        
        # Embedding layer: pad with random init for new tokens
        elif 'embedding' in k and len(new_shape) == 2:
            old_vocab, embed_dim = prev_shape
            new_vocab, _ = new_shape
            # Copy old embeddings
            min_vocab = min(old_vocab, new_vocab)
            new_dict[k][:min_vocab] = prev_dict[k][:min_vocab].clone()
            transferred_params += min_vocab * embed_dim
            if verbose:
                print(f"  {k}: Transferred {min_vocab}/{new_vocab} embeddings, rest random")
        
        # Output layer (linear2): pad/crop output dimension
        elif 'linear2' in k and len(new_shape) == 2:
            hidden, old_out = prev_shape
            _, new_out = new_shape
            min_out = min(old_out, new_out)
            new_dict[k][:min_out] = prev_dict[k][:min_out].clone()
            transferred_params += min_out * hidden
            if verbose:
                print(f"  {k}: Transferred {min_out}/{new_out} outputs, rest random")
        
        else:
            if verbose:
                print(f"  {k}: Shape mismatch {prev_shape} -> {new_shape}, keeping random")
    
    new_model.load_state_dict(new_dict)
    transfer_pct = 100 * transferred_params / total_params
    print(f"✅ Transferred {transferred_params:,}/{total_params:,} parameters ({transfer_pct:.1f}%)")
    return new_model

def train(train_dataset, test_dataset, params, model=None, verbose=True):
    """Train model for params.n_batches_per_stage batches."""
    all_models = []
    if model is None:
        model = MLP(params).to(params.device)
    
    optimizer = torch.optim.Adam(
        model.parameters(), weight_decay=params.weight_decay, lr=params.lr
    )
    loss_fn = torch.nn.CrossEntropyLoss()
    train_loader = DataLoader(train_dataset, batch_size=params.batch_size, shuffle=True)
    
    print_every = params.n_batches_per_stage // params.print_times
    checkpoint_every = None
    if params.n_save_model_checkpoints > 0:
        checkpoint_every = params.n_batches_per_stage // params.n_save_model_checkpoints
    
    loss_data = []
    if verbose:
        pbar = tqdm(total=params.n_batches_per_stage, desc="Training")
    
    for i in range(params.n_batches_per_stage):
        batch = next(iter(train_loader))
        X, Y = batch
        X, Y = X.to(params.device), Y.to(params.device)
        
        optimizer.zero_grad()
        out = model(X)
        loss = loss_fn(out, Y)
        loss.backward()
        optimizer.step()
        
        if checkpoint_every and (i + 1) % checkpoint_every == 0:
            all_models += [deepcopy(model)]
        
        if (i + 1) % print_every == 0:
            val_acc, val_loss = test(model, test_dataset, params.device)
            train_acc, train_loss = test(model, train_dataset, params.device)
            loss_data.append({
                "batch": i + 1,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc,
            })
            if verbose:
                pbar.set_postfix({
                    "train_loss": f"{train_loss:.4f}",
                    "train_acc": f"{train_acc:.4f}",
                    "val_loss": f"{val_loss:.4f}",
                    "val_acc": f"{val_acc:.4f}",
                })
                pbar.update(print_every)
    
    if verbose:
        pbar.close()
    
    df = pd.DataFrame(loss_data)
    train_acc, train_loss = test(model, train_dataset, params.device)
    val_acc, val_loss = test(model, test_dataset, params.device)
    
    if verbose:
        print(f"Final Train Acc: {train_acc:.4f} | Final Train Loss: {train_loss:.4f}")
        print(f"Final Val Acc: {val_acc:.4f} | Final Val Loss: {val_loss:.4f}")
    
    return all_models, df

def set_seed(seed):
    """Set all random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_all_pairs(p):
    pairs = []
    for i in range(p):
        for j in range(p):
            pairs.append((i, j))
    return pairs

def make_dataset(p):
    data = []
    pairs = get_all_pairs(p)
    for a, b in pairs:
        data.append((torch.tensor([a, b]), torch.tensor((a + b) % p)))
    return data

def train_test_split(dataset, train_split_proportion, seed):
    """Split dataset deterministically based on seed."""
    l = len(dataset)
    train_len = int(train_split_proportion * l)
    idx = list(range(l))
    
    # Use local RNG to avoid polluting global state
    rng = random.Random(seed)
    rng.shuffle(idx)
    
    train_idx = idx[:train_len]
    test_idx = idx[train_len:]
    return [dataset[i] for i in train_idx], [dataset[i] for i in test_idx]

print("Model and utility functions defined!")

## 4. Create Datasets

Prepare datasets once (they'll be reused across seeds)

In [ ]:
# Prepare datasets for each prime in curriculum and for direct training
curriculum_datasets = {}
curriculum_splits = {}

for p in CURRICULUM_NUMS:
    dataset = make_dataset(p)
    train_data, test_data = train_test_split(dataset, 0.4, seed=0)  # Same split for all
    curriculum_datasets[p] = dataset
    curriculum_splits[p] = (train_data, test_data)
    print(f"mod-{p}: {len(train_data)} train, {len(test_data)} test")

direct_dataset = make_dataset(DIRECT_MODULUS)
direct_train_data, direct_test_data = train_test_split(direct_dataset, 0.4, seed=0)
print(f"\nDirect (mod-{DIRECT_MODULUS}): {len(direct_train_data)} train, {len(direct_test_data)} test")

## 5. Run Curriculum Learning Across Multiple Seeds

**Key Changes:**
- Run 5 independent seeds
- Use proper weight transfer with padding/cropping
- Store results for statistical analysis

In [ ]:
curriculum_results = {}  # {seed: {stage: {checkpoints, df}}}

for seed in SEEDS:
    print("\n" + "="*80)
    print(f"CURRICULUM LEARNING - SEED {seed}")
    print("="*80)
    
    set_seed(seed)
    curriculum_results[seed] = {}
    prev_model = None
    
    for stage_idx, p in enumerate(CURRICULUM_NUMS):
        print(f"\nStage {stage_idx+1}/{len(CURRICULUM_NUMS)}: mod-{p}")
        
        params = ExperimentParams()
        params.p = p
        params.device = DEVICE
        
        train_data, test_data = curriculum_splits[p]
        train_data = [(x.to(params.device), y.to(params.device)) for x, y in train_data]
        test_data = [(x.to(params.device), y.to(params.device)) for x, y in test_data]
        
        # Transfer weights from previous stage
        if prev_model is not None:
            model = MLP(params)
            model = transfer_weights_with_padding(model, prev_model, verbose=False)
            model = model.to(params.device)
        else:
            model = None
        
        checkpoints, df = train(train_data, test_data, params, model=model, verbose=True)
        curriculum_results[seed][p] = {'checkpoints': checkpoints, 'df': df}
        prev_model = checkpoints[-1]

print("\n" + "="*80)
print("✅ Curriculum learning complete for all seeds!")
print("="*80)

## 6. Run Direct Training Across Multiple Seeds

**Key Change:** Direct training now gets 200k batches (same as curriculum total)

In [ ]:
direct_results = {}  # {seed: {checkpoints, df}}

for seed in SEEDS:
    print("\n" + "="*80)
    print(f"DIRECT TRAINING - SEED {seed}")
    print("="*80)
    
    set_seed(seed)
    
    params = ExperimentParams()
    params.p = DIRECT_MODULUS
    params.device = DEVICE
    params.n_batches_per_stage = DIRECT_TOTAL_BATCHES  # 200k batches!
    
    train_data = [(x.to(params.device), y.to(params.device)) for x, y in direct_train_data]
    test_data = [(x.to(params.device), y.to(params.device)) for x, y in direct_test_data]
    
    checkpoints, df = train(train_data, test_data, params, model=None, verbose=True)
    direct_results[seed] = {'checkpoints': checkpoints, 'df': df}

print("\n" + "="*80)
print("✅ Direct training complete for all seeds!")
print("="*80)

## 7. Plot Accuracy and Loss with Confidence Intervals

**Key Change:** Show mean ± std across seeds

In [ ]:
def aggregate_metrics(results_dict, modulus_key):
    """Aggregate metrics across seeds, return mean and std."""
    all_dfs = [results_dict[seed][modulus_key]['df'] for seed in SEEDS]
    
    # Ensure all have same length
    min_len = min(len(df) for df in all_dfs)
    
    metrics = {}
    for col in ['train_acc', 'val_acc', 'train_loss', 'val_loss']:
        values = np.array([df[col].values[:min_len] for df in all_dfs])
        metrics[col + '_mean'] = np.mean(values, axis=0)
        metrics[col + '_std'] = np.std(values, axis=0)
    
    return metrics, min_len

# Plot final curriculum stage vs direct
curriculum_final_metrics, curr_len = aggregate_metrics(curriculum_results, CURRICULUM_NUMS[-1])
direct_metrics, direct_len = aggregate_metrics(direct_results, None)  # Direct doesn't use modulus key

# Need to handle direct_results structure
all_direct_dfs = [direct_results[seed]['df'] for seed in SEEDS]
min_direct_len = min(len(df) for df in all_direct_dfs)
direct_metrics = {}
for col in ['train_acc', 'val_acc', 'train_loss', 'val_loss']:
    values = np.array([df[col].values[:min_direct_len] for df in all_direct_dfs])
    direct_metrics[col + '_mean'] = np.mean(values, axis=0)
    direct_metrics[col + '_std'] = np.std(values, axis=0)

# Plot accuracy comparison
fig, ax = plt.subplots(figsize=(12, 6))
x_curr = np.arange(len(curriculum_final_metrics['val_acc_mean']))
x_direct = np.arange(len(direct_metrics['val_acc_mean']))

# Curriculum
ax.plot(x_curr, curriculum_final_metrics['val_acc_mean'], label='Curriculum: Test Acc', color='blue')
ax.fill_between(x_curr, 
                curriculum_final_metrics['val_acc_mean'] - curriculum_final_metrics['val_acc_std'],
                curriculum_final_metrics['val_acc_mean'] + curriculum_final_metrics['val_acc_std'],
                alpha=0.3, color='blue')

# Direct
ax.plot(x_direct, direct_metrics['val_acc_mean'], label='Direct: Test Acc', color='orange')
ax.fill_between(x_direct,
                direct_metrics['val_acc_mean'] - direct_metrics['val_acc_std'],
                direct_metrics['val_acc_mean'] + direct_metrics['val_acc_std'],
                alpha=0.3, color='orange')

ax.set_xlabel('Checkpoint')
ax.set_ylabel('Test Accuracy')
ax.set_title(f'Curriculum vs Direct: Test Accuracy (mean ± std, n={len(SEEDS)} seeds)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_and_show(fig, 'curriculum_vs_direct_accuracy_with_ci.png')

# Plot loss comparison
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(x_curr, curriculum_final_metrics['val_loss_mean'], label='Curriculum: Test Loss', color='blue')
ax.fill_between(x_curr,
                curriculum_final_metrics['val_loss_mean'] - curriculum_final_metrics['val_loss_std'],
                curriculum_final_metrics['val_loss_mean'] + curriculum_final_metrics['val_loss_std'],
                alpha=0.3, color='blue')

ax.plot(x_direct, direct_metrics['val_loss_mean'], label='Direct: Test Loss', color='orange')
ax.fill_between(x_direct,
                direct_metrics['val_loss_mean'] - direct_metrics['val_loss_std'],
                direct_metrics['val_loss_mean'] + direct_metrics['val_loss_std'],
                alpha=0.3, color='orange')

ax.set_xlabel('Checkpoint')
ax.set_ylabel('Test Loss')
ax.set_title(f'Curriculum vs Direct: Test Loss (mean ± std, n={len(SEEDS)} seeds)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_yscale('log')  # Log scale for loss
plt.tight_layout()
save_and_show(fig, 'curriculum_vs_direct_loss_with_ci.png')

## 8. LLC Hyperparameters - Adaptive by Model Size

**Key Change:** Scale LLC hyperparameters based on model complexity (vocabulary size)

In [ ]:
def get_llc_hyperparams(vocab_size):
    """Get LLC hyperparameters scaled by model size.
    
    Larger vocabulary = more parameters in embedding/output layers,
    may need adjusted localization and learning rate.
    """
    base_lr = 3e-3
    base_gamma = 5.0
    
    # Scale learning rate down slightly for larger models
    lr_scale = np.sqrt(8 / vocab_size)  # Reference: mod-8
    lr = base_lr * lr_scale
    
    # Scale localization up slightly for larger models
    gamma_scale = np.sqrt(vocab_size / 8)
    gamma = base_gamma * gamma_scale
    
    return {
        'lr': lr,
        'nbeta': 2.0,  # Keep constant
        'gamma': gamma,
        'num_chains': 3,  # Use 3 chains for reliability
        'num_draws': 1000,
    }

# Show hyperparameters for each modulus
print("LLC Hyperparameters by modulus:")
print("="*60)
for p in CURRICULUM_NUMS:
    hp = get_llc_hyperparams(p)
    print(f"mod-{p:2d}: lr={hp['lr']:.4f}, γ={hp['gamma']:.2f}, chains={hp['num_chains']}")
print("="*60)

## 9. Estimate LLC for Curriculum (Single Seed First)

**Key Changes:**
- Use 3 chains instead of 1
- Adaptive hyperparameters by model size

**Note:** Running LLC for all seeds × all checkpoints would take very long.
We'll do seed 0 fully, then sample other seeds for statistical testing.

In [ ]:
# Estimate LLC for seed 0 curriculum (all checkpoints, all stages)
seed = 0
curriculum_llcs_seed0 = {}

print("\n" + "="*80)
print(f"Estimating LLC for CURRICULUM (seed {seed})")
print("="*80)

for stage_idx, p in enumerate(CURRICULUM_NUMS):
    print(f"\nStage {stage_idx+1}/{len(CURRICULUM_NUMS)}: mod-{p}")
    
    params = ExperimentParams()
    params.p = p
    train_data, _ = curriculum_splits[p]
    loader = DataLoader(train_data, batch_size=params.batch_size, shuffle=True)
    
    checkpoints = curriculum_results[seed][p]['checkpoints']
    hp = get_llc_hyperparams(p)
    
    print(f"  Estimating LLC for {len(checkpoints)} checkpoints...")
    print(f"  Hyperparams: lr={hp['lr']:.4f}, γ={hp['gamma']:.2f}, chains={hp['num_chains']}")
    
    stage_llcs = []
    for checkpoint_idx, model_checkpoint in enumerate(checkpoints):
        if (checkpoint_idx + 1) % 20 == 0 or checkpoint_idx == 0 or checkpoint_idx == len(checkpoints) - 1:
            print(f"    Checkpoint {checkpoint_idx+1}/{len(checkpoints)}")
        
        llc_stats = estimate_learning_coeff_with_summary(
            model_checkpoint,
            loader=loader,
            evaluate=evaluate_ce,
            sampling_method=SGLD,
            optimizer_kwargs=dict(lr=hp['lr'], nbeta=hp['nbeta'], localization=hp['gamma']),
            num_chains=hp['num_chains'],
            num_draws=hp['num_draws'],
            device=DEVICE,
            online=False,
        )
        stage_llcs.append(llc_stats)
    
    curriculum_llcs_seed0[p] = stage_llcs
    llc_values = [llc['llc/mean'] for llc in stage_llcs]
    print(f"  Completed: LLC range [{min(llc_values):.2f}, {max(llc_values):.2f}]")

print("\n" + "="*80)
print("✅ Curriculum LLC estimation complete (seed 0)!")
print("="*80)

## 10. Estimate LLC for Direct Training (Single Seed)

Same approach as curriculum

In [ ]:
# Estimate LLC for seed 0 direct training
seed = 0

print("\n" + "="*80)
print(f"Estimating LLC for DIRECT TRAINING (seed {seed}, mod-{DIRECT_MODULUS})")
print("="*80)

params = ExperimentParams()
params.p = DIRECT_MODULUS
loader = DataLoader(direct_train_data, batch_size=params.batch_size, shuffle=True)

checkpoints = direct_results[seed]['checkpoints']
hp = get_llc_hyperparams(DIRECT_MODULUS)

print(f"Total checkpoints: {len(checkpoints)}")
print(f"Hyperparams: lr={hp['lr']:.4f}, γ={hp['gamma']:.2f}, chains={hp['num_chains']}")

direct_llcs_seed0 = []
for i, model_checkpoint in enumerate(checkpoints):
    if (i + 1) % 20 == 0 or i == 0 or i == len(checkpoints) - 1:
        print(f"  Checkpoint {i+1}/{len(checkpoints)}")
    
    llc_stats = estimate_learning_coeff_with_summary(
        model_checkpoint,
        loader=loader,
        evaluate=evaluate_ce,
        sampling_method=SGLD,
        optimizer_kwargs=dict(lr=hp['lr'], nbeta=hp['nbeta'], localization=hp['gamma']),
        num_chains=hp['num_chains'],
        num_draws=hp['num_draws'],
        device=DEVICE,
        online=False,
    )
    direct_llcs_seed0.append(llc_stats)

llc_values = [llc['llc/mean'] for llc in direct_llcs_seed0]
print(f"\nCompleted! LLC range: [{min(llc_values):.2f}, {max(llc_values):.2f}]")
print("="*80)

## 11. Plot LLC Trajectories with Confidence Intervals

Visualize LLC evolution across training

In [ ]:
# Plot curriculum LLC vs accuracy for final stage
fig, ax1 = plt.subplots(figsize=(14, 6))
ax2 = ax1.twinx()

# Accuracy from all seeds
x = np.arange(len(curriculum_final_metrics['val_acc_mean']))
ax1.plot(x, curriculum_final_metrics['val_acc_mean'], label='Test Acc (mean)', color='blue')
ax1.fill_between(x,
                 curriculum_final_metrics['val_acc_mean'] - curriculum_final_metrics['val_acc_std'],
                 curriculum_final_metrics['val_acc_mean'] + curriculum_final_metrics['val_acc_std'],
                 alpha=0.3, color='blue')

# LLC from seed 0
llc_values = [llc['llc/mean'] for llc in curriculum_llcs_seed0[CURRICULUM_NUMS[-1]]]
ax2.plot(llc_values, color='green', label='LLC (seed 0)', linewidth=2)

ax1.set_xlabel('Checkpoint')
ax1.set_ylabel('Test Accuracy', color='blue')
ax2.set_ylabel('LLC (λ̂)', color='green')
ax1.set_title(f'Curriculum (mod-{CURRICULUM_NUMS[-1]}): LLC vs Accuracy')
ax1.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='green')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
plt.tight_layout()
save_and_show(fig, 'curriculum_llc_vs_accuracy.png')

# Plot direct LLC vs accuracy
fig, ax1 = plt.subplots(figsize=(14, 6))
ax2 = ax1.twinx()

x = np.arange(len(direct_metrics['val_acc_mean']))
ax1.plot(x, direct_metrics['val_acc_mean'], label='Test Acc (mean)', color='blue')
ax1.fill_between(x,
                 direct_metrics['val_acc_mean'] - direct_metrics['val_acc_std'],
                 direct_metrics['val_acc_mean'] + direct_metrics['val_acc_std'],
                 alpha=0.3, color='blue')

llc_values = [llc['llc/mean'] for llc in direct_llcs_seed0]
ax2.plot(llc_values, color='green', label='LLC (seed 0)', linewidth=2)

ax1.set_xlabel('Checkpoint')
ax1.set_ylabel('Test Accuracy', color='blue')
ax2.set_ylabel('LLC (λ̂)', color='green')
ax1.set_title(f'Direct (mod-{DIRECT_MODULUS}): LLC vs Accuracy')
ax1.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='green')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
plt.tight_layout()
save_and_show(fig, 'direct_llc_vs_accuracy.png')

# Comparison plot
fig = plt.figure(figsize=(14, 6))
curriculum_llc = [llc['llc/mean'] for llc in curriculum_llcs_seed0[CURRICULUM_NUMS[-1]]]
direct_llc = [llc['llc/mean'] for llc in direct_llcs_seed0]

plt.plot(curriculum_llc, label=f'Curriculum (mod-{CURRICULUM_NUMS[-1]})', linewidth=2, color='blue')
plt.plot(direct_llc, label=f'Direct (mod-{DIRECT_MODULUS})', linewidth=2, color='orange', linestyle='--')

plt.xlabel('Checkpoint', fontsize=12)
plt.ylabel('LLC (λ̂)', fontsize=12)
plt.title('LLC Trajectories: Curriculum vs Direct (seed 0, equalized budgets)', fontsize=13)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
save_and_show(fig, 'llc_comparison_curriculum_vs_direct.png')

## 12. Statistical Significance Testing

Test if curriculum vs direct differences are statistically significant

In [ ]:
# Compare final test accuracy across seeds
curriculum_final_accs = []
direct_final_accs = []

for seed in SEEDS:
    curr_df = curriculum_results[seed][CURRICULUM_NUMS[-1]]['df']
    curriculum_final_accs.append(curr_df['val_acc'].iloc[-1])
    
    direct_df = direct_results[seed]['df']
    direct_final_accs.append(direct_df['val_acc'].iloc[-1])

# T-test
t_stat, p_value = stats.ttest_ind(curriculum_final_accs, direct_final_accs)

print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE TESTING")
print("="*80)
print(f"\nFinal Test Accuracy Comparison (n={len(SEEDS)} seeds):")
print(f"  Curriculum: {np.mean(curriculum_final_accs):.4f} ± {np.std(curriculum_final_accs):.4f}")
print(f"  Direct:     {np.mean(direct_final_accs):.4f} ± {np.std(direct_final_accs):.4f}")
print(f"\nTwo-sample t-test:")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value:     {p_value:.4f}")

if p_value < 0.05:
    print(f"\n✅ Difference is statistically significant (p < 0.05)")
else:
    print(f"\n⚠️  Difference is NOT statistically significant (p >= 0.05)")

# Effect size (Cohen's d)
pooled_std = np.sqrt((np.var(curriculum_final_accs) + np.var(direct_final_accs)) / 2)
cohens_d = (np.mean(curriculum_final_accs) - np.mean(direct_final_accs)) / pooled_std
print(f"  Cohen's d:   {cohens_d:.4f}")

if abs(cohens_d) < 0.2:
    effect = "negligible"
elif abs(cohens_d) < 0.5:
    effect = "small"
elif abs(cohens_d) < 0.8:
    effect = "medium"
else:
    effect = "large"
print(f"  Effect size: {effect}")
print("="*80)

# Visualize distributions
fig, ax = plt.subplots(figsize=(10, 6))
positions = [1, 2]
data = [curriculum_final_accs, direct_final_accs]
labels = ['Curriculum', 'Direct']

bp = ax.boxplot(data, positions=positions, labels=labels, patch_artist=True)
for patch, color in zip(bp['boxes'], ['blue', 'orange']):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

# Add individual points
for i, (pos, vals) in enumerate(zip(positions, data)):
    x = np.random.normal(pos, 0.04, len(vals))
    ax.scatter(x, vals, alpha=0.6, s=50, color=['blue', 'orange'][i])

ax.set_ylabel('Final Test Accuracy')
ax.set_title(f'Final Test Accuracy Distribution (n={len(SEEDS)} seeds)\np-value = {p_value:.4f}')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
save_and_show(fig, 'final_accuracy_distribution.png')

## 13. Summary Statistics

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT SUMMARY")
print("="*80)

print("\n🔧 Experimental Setup:")
print(f"  • Curriculum stages: {CURRICULUM_NUMS}")
print(f"  • Total batches: {CURRICULUM_TOTAL_BATCHES:,} (both curriculum and direct)")
print(f"  • Seeds: {len(SEEDS)}")
print(f"  • LLC chains: 3")
print(f"  • Weight transfer: Pad/crop (not random reinitialization)")

print("\n📊 Final Results (mean ± std across seeds):")
print(f"  Curriculum final test acc: {np.mean(curriculum_final_accs):.4f} ± {np.std(curriculum_final_accs):.4f}")
print(f"  Direct final test acc:     {np.mean(direct_final_accs):.4f} ± {np.std(direct_final_accs):.4f}")
print(f"  Difference:                {np.mean(curriculum_final_accs) - np.mean(direct_final_accs):.4f}")
print(f"  Statistical significance:  p = {p_value:.4f} ({'sig' if p_value < 0.05 else 'not sig'})")

print("\n📈 LLC Statistics (seed 0):")
for stage_idx, p in enumerate(CURRICULUM_NUMS):
    llc_vals = [llc['llc/mean'] for llc in curriculum_llcs_seed0[p]]
    print(f"  Curriculum stage {stage_idx+1} (mod-{p}):")
    print(f"    Initial LLC: {llc_vals[0]:.2f}")
    print(f"    Final LLC:   {llc_vals[-1]:.2f}")

direct_llc_vals = [llc['llc/mean'] for llc in direct_llcs_seed0]
print(f"\n  Direct (mod-{DIRECT_MODULUS}):")
print(f"    Initial LLC: {direct_llc_vals[0]:.2f}")
print(f"    Final LLC:   {direct_llc_vals[-1]:.2f}")

print("\n" + "="*80)
print("✅ EXPERIMENT COMPLETE")
print("="*80)